In [19]:
pip install requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [3]:
pip install elasticsearch>=8.17.0,<9.0.0

/bin/bash: 9.0.0: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [27]:
from elasticsearch import Elasticsearch

In [28]:
es_client = Elasticsearch('http://localhost:9200')

In [29]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body = index_settings)

Index 'course-questions' already exists. Skipping creation.


In [30]:
from tqdm.auto import tqdm 

In [31]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 287.80it/s]


In [32]:
elastic_query = 'How do execute a command on a Kubernetes pod?'

In [33]:
def elastic_search(elastic_query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": elastic_query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    } 

    response = es_client.search(index = index_name, body = search_query)
    return response

In [34]:
search = elastic_search(elastic_query)

In [35]:
print(search['hits']['hits'][0]['_score'])

32.00351


In [36]:
query = "How do copy a file to a Docker container?"

In [42]:
def elastic_filtering(elastic_query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": elastic_query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    } 

    response = es_client.search(index = index_name, body = search_query)
    return response['hits']['hits']

In [43]:
filtering = elastic_filtering(query)

In [44]:
print(filtering)

[{'_index': 'course-questions', '_id': 's68uWpcBg1Y_f0M-nLMu', '_score': 73.50458, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'Z68xWpcBg1Y_f0M-6rdf', '_score': 73.50458, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', '

In [45]:
print(filtering[2])

{'_index': 'course-questions', '_id': '0q8uWpcBg1Y_f0M-nLOl', '_score': 66.94253, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'machine-learning-zoomcamp'}}


In [63]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_item_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    for doc in search_results:
        context += context_item_template.format(question=doc['_source']['question'], text=doc['_source']['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [64]:
 prompt = build_prompt(query, filtering)

In [65]:
len(prompt)

1363

In [66]:
pip install tiktoken


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [67]:
import tiktoken

In [68]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [69]:
tokens = encoding.encode(prompt)

In [70]:
print(len(tokens))

305
